In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_out = pd.read_csv("test.csv")
test['SalePrice'] = 0

In [3]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,0
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,0
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,0
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,0
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,0


In [4]:
def correlation(ds):
    numeric_features = train.select_dtypes(include=[np.number]) #catch numeric columns
    numeric_features.corr()['SalePrice'].sort_values(ascending = False)[:10]

In [5]:
def data_clean(ds,test):
    #print(ds.head())
    ds['MSZoning'].replace({'C (all)': 0, 'RM':1, 'RH':2, 'RL':3, 'FV':4}, inplace = True) 
    numeric_features = ds.select_dtypes(include=[np.number])
    corr = numeric_features.corr()
    #print (corr['SalePrice'].sort_values(ascending=False))
    a = numeric_features.corr()['SalePrice'].sort_values(ascending=False)[:10] #pega as 10 maiores correlações
    print(a)
    index = a.index #retira o index delas 
    ds[index] = ds[index].fillna(0)
    ds = ds[index]
    ds = ds.dropna()
    
    test['MSZoning'].replace({'C (all)': 0, 'RM':1, 'RH':2, 'RL':3, 'FV':4}, inplace = True)
    test[index] = test[index].fillna(0)
    test = test[index]
    test = test.dropna()
    #print(ds.isnull().sum())
    
    return ds,test

In [6]:
###Teste
(train,test) = data_clean(train,test)
#test = data_clean(test)

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
Name: SalePrice, dtype: float64


In [7]:
train.groupby('GarageCars').median()

,SalePrice,OverallQual,GrLivArea,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt
GarageCars,,,,,,,,,
0,100000.0,5.0,1080.0,0.0,780.0,864.0,1.0,6.0,1940.0
1,128000.0,5.0,1148.0,288.0,876.0,949.0,1.0,6.0,1954.0
2,177750.0,6.0,1501.5,506.0,1022.5,1128.0,2.0,6.0,1982.5
3,295000.0,8.0,1962.0,818.0,1572.0,1584.0,2.0,8.0,2005.0
4,200000.0,5.0,1622.0,864.0,1248.0,1338.0,1.0,8.0,1969.0


In [8]:
train.isnull().sum()

SalePrice       0
OverallQual     0
GrLivArea       0
GarageCars      0
GarageArea      0
TotalBsmtSF     0
1stFlrSF        0
FullBath        0
TotRmsAbvGrd    0
YearBuilt       0
dtype: int64

In [9]:
'''corr = numeric_features.corr()
print (corr['SalePrice'].sort_values(ascending=False)[:10], '\n')
print (corr['SalePrice'].sort_values(ascending=False)[-5:])'''

"corr = numeric_features.corr()\nprint (corr['SalePrice'].sort_values(ascending=False)[:10], '\n')\nprint (corr['SalePrice'].sort_values(ascending=False)[-5:])"

In [10]:
'''print (corr['SalePrice'].sort_values(ascending=False))'''

"print (corr['SalePrice'].sort_values(ascending=False))"

In [11]:
'''a = numeric_features.corr()['SalePrice'].sort_values(ascending=False)[:15] #pega as 10 maiores correlações
index = a.index #retira o index delas'''

"a = numeric_features.corr()['SalePrice'].sort_values(ascending=False)[:15] #pega as 10 maiores correlações\nindex = a.index #retira o index delas"

In [12]:
'''ds = train[index] #utiliza apenas as colunas das melhores correlações
ds'''

'ds = train[index] #utiliza apenas as colunas das melhores correlações\nds'

In [13]:
#ds.isnull().sum()
train.isnull().sum()

SalePrice       0
OverallQual     0
GrLivArea       0
GarageCars      0
GarageArea      0
TotalBsmtSF     0
1stFlrSF        0
FullBath        0
TotRmsAbvGrd    0
YearBuilt       0
dtype: int64

In [14]:
X = train.drop(['SalePrice'], axis='columns')
y = train['SalePrice']

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
#Random Forest Classifier
model2=RandomForestClassifier(n_estimators =100)
model2.fit(X_train,y_train)
pred_2=model2.predict(X_test)
rcf=accuracy_score(pred_2,y_test);
print(rcf)   

0.00684931506849315


In [17]:
#Logistic Regression
model1 = LogisticRegression(solver='lbfgs', max_iter=1000)
model1.fit(X_train, y_train)
pred = model1.predict(X_test)
log_acc = accuracy_score(pred, y_test)
print(log_acc)

0.0136986301369863


C:\Users\Danilo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
#index = index.drop('SalePrice')

In [21]:
test.drop('SalePrice',axis = 'columns', inplace = True)

In [22]:
model2.fit(X,y)
pred = model2.predict(test)

In [ ]:
test[index].isnull().sum()

In [24]:
result = pd.DataFrame()
result['Id'] = test_out.Id
result['SalePrice'] = pred

In [ ]:
result.head(5)

In [ ]:
result.to_csv('sub.csv', index = False)

In [ ]:
n = pd.DataFrame()
n['MSZoning'] = train.MSZoning
n['SalePrice'] = train.SalePrice
n.groupby('MSZoning').mean()['SalePrice'].sort_values(ascending=True)

In [ ]:
test = data_clean(test)

In [ ]:
test = test.drop('SalePrice',axis = 'columns', inplace = True)


In [ ]:
print(test)

In [ ]:
train.columns